목적 : 탐색적 데이터 분석 EDA
=====================
목표 : 기초통계량, 상관분석(+유의성 검정),Q-Qplot
-------------------------    

import modules

In [12]:
import pandas as pd
import numpy as np
from collections import Counter

%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
import statsmodels.api as sm

import platform
if platform.system() == "Darwin":           # MAC인 경우
    plt.rc('font', family ="AppleGothic")
else:
    plt.rc('font',family='Malgun Gothic')

#멧플로립에서 음수를 표시
plt.rc('axes',unicode_minus=False)

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')


import scipy.stats as stats
from scipy.stats import shapiro, pearsonr
from itertools import combinations
from sklearn.model_selection import train_test_split

환경설정

In [13]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

# pd.set_option('display.max_columns', None)
pd.reset_option('display.max_columns')

pd.set_option('display.max_seq_items', None)

pd.set_option('float_format', '{:.3f}'.format) # 소수점 3개까지 출력

load file

In [14]:
unlisted = pd.read_csv("./비상장후보피처.csv")
# 영업년수와 상장년수가 있더라

In [15]:
unlisted.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '부실판단', '도입기', '성숙기', '성장기', '쇠퇴기'],
      dtype='object')

# train test split

In [16]:
X = unlisted.drop('부실판단', axis=1)
y = unlisted['부실판단']
# 데이터를 학습용과 테스트용으로 나눔
unlisted_X_train, unlisted_X_test, unlisted_y_train, unlisted_y_test = train_test_split(X, y, test_size= 1 / (np.exp(1) + 1) , random_state=911, stratify=y)

In [17]:
unlisted_X_train['부실판단'] = unlisted_y_train

In [18]:
len(X)

20707

In [19]:
len(unlisted_X_train)

15138

In [20]:
len(unlisted_X_test)

5569

# 범주 분할 1현금흐름 / 2안정성 / 2수익성 / 3유동성 / 4성장성 / 5생산성 / 6활동성 / 7시장지표 및 기타

In [21]:
# 비상장 all data는 unlisted
# 비상장 label들

labels = unlisted_X_train[
    [
        '거래소코드',
        '회계년도',
        '부실판단',
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기'
    ]
]

# 비상장 피처
features = unlisted_X_train.drop(
    labels=[
        '회사명',
        '거래소코드',
        '회계년도',        
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'        
    ],
    axis=1,    
)

# 1현금흐름 4개
cash_ft1 = unlisted_X_train[
    [
        'CASH FLOW 대 부채비율',
        'CASH FLOW 대 차입금비율',
        'CASH FLOW 대 총자본비율',
        'CASH FLOW 대 매출액비율',
        '부실판단'
    ]
]

# 2안정성 9개
stability_ft1 = unlisted_X_train[
    [
        '유동비율',
        '당좌비율',
        '부채비율',
        '유동부채비율',
        '차입금의존도',
        '순운전자본비율',
        '현금비율',
        '유동자산구성비율',
        '자기자본구성비율',
        '부실판단'
    ]
]

# 3수익성 8개
profitability_ft1 = unlisted_X_train[
    [
        '자기자본순이익률',
        '경영자본순이익률',
        '매출액순이익률',
        '총자본사업이익률',
        '총자본영업이익률',
        '매출액영업이익률',
        '금융비용부담률',
        'CROE',
        '부실판단'
    ]
]

# 4성장성 6개
growth_ft1 = unlisted_X_train[
    [
        '총자산증가율',
        '총자본증가율',
        '유동자산증가율',
        '비유동자산증가율',
        '매출액증가율',
        '유형자산증가율',
        '부실판단'
    ]
]

# 5생산성 6개
productivity_ft1 = unlisted_X_train[
    [
        '부가가치(백만원)',
        '총자본투자효율',
        '설비투자효율',
        '부가가치율',
        '이윤분배율',
        '자본분배율',
        '부실판단'
    ]
]

# 6활동성 8개
activity_ft1 = unlisted_X_train[
    [
        '총자산회전율',
        '매출채권회전률',
        '당좌자산회전률',
        '유형자산회전율',
        '경영자본회전률',
        '총자본회전률',
        '비유동자산회전률',
        '유동자산회전률',
        '부실판단'
    ]
]

# 7기타
etc_ft1 = unlisted_X_train[
    [
        '영업년수', 
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'
    ]
]

# describe

all

In [22]:
unlisted_X_train['부실판단'] = unlisted_y_train

In [23]:
unlisted_X_train

,회사명,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,...,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,영업년수,도입기,성숙기,성장기,쇠퇴기,부실판단
7716,(주)동남코리아,70118,2022,-18.740,-39.580,-3.410,-27.330,26.090,17.940,22.200,...,0.310,0.130,0.140,3.590,30.000,1,0,0,0,0.000
4739,피에스케이홀딩스(주),20666,2019,130.010,22131.400,10.600,36.690,1385.060,1230.540,8.880,...,0.970,0.300,0.400,1.200,38.000,0,1,0,0,0.000
12685,금용해양산업주식회사,63939,2022,121.510,368.960,29.460,46.110,159.900,159.900,32.010,...,0.700,0.700,1.010,2.230,14.000,0,1,0,0,0.000
17405,주식회사대일,53349,2014,-2.390,-3.210,-1.850,-2.010,87.410,23.480,341.770,...,0.960,0.940,2.000,1.760,22.000,1,0,0,0,1.000
18617,주식회사삼호프레스,83444,2018,-1.480,-2.030,-0.910,-2.540,71.900,59.910,160.760,...,0.500,0.350,0.420,2.250,22.000,0,0,0,1,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13013,농업회사법인한만두식품주식회사,82850,2022,6.720,8.850,5.280,5.950,95.860,63.560,367.770,...,0.920,0.920,1.510,2.320,13.000,0,0,1,0,0.000
12657,금성냉장주식회사,29681,2014,-0.530,-1.020,-0.440,-0.520,28.600,13.240,484.900,...,0.860,0.830,1.040,4.200,24.000,0,1,0,0,1.000
11925,(주)노비아레텍,70975,2022,-7.670,-11.400,-5.630,-4.340,100.960,38.140,276.910,...,1.360,1.360,4.550,1.940,12.000,1,0,0,0,1.000
10995,새한에프앤비(주),13761,2014,-9.190,-10.580,-6.360,-17.210,45.060,45.060,224.730,...,0.710,0.360,0.480,1.370,31.000,0,0,0,1,1.000


In [24]:
unlisted_X_train.describe()

,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,...,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,영업년수,도입기,성숙기,성장기,쇠퇴기,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,...,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,67773.011,2019.874,19.314,724.234,3.241,-22.741,431.245,358.475,1378.936,983.416,...,1.119,0.996,22.988,2.830,21.322,0.218,0.368,0.191,0.223,0.361
std,23619.347,2.797,1261.272,31970.320,37.792,539.239,5347.663,5224.565,35986.322,30885.126,...,1.590,1.404,375.217,5.156,11.151,0.413,0.482,0.393,0.416,0.480
min,10037.000,2014.000,-86467.650,-323703.660,-3388.620,-21110.050,0.070,0.070,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,52329.000,2018.000,-4.480,-5.120,-2.880,-4.890,56.050,31.620,60.845,36.860,...,0.340,0.280,0.490,1.060,13.000,0.000,0.000,0.000,0.000,0.000
50%,70853.000,2022.000,2.920,1.510,1.860,2.360,108.715,72.700,163.855,102.565,...,0.820,0.730,1.460,1.990,20.000,0.000,0.000,0.000,0.000,0.000
75%,87046.750,2022.000,14.328,17.380,7.387,10.777,197.823,150.323,373.438,239.108,...,1.430,1.300,3.757,3.340,27.000,0.000,1.000,0.000,0.000,1.000
max,101635.000,2022.000,113999.840,3194164.340,1274.750,21579.540,447760.110,447760.110,3385006.150,2905990.690,...,106.800,92.050,32610.280,344.060,110.000,1.000,1.000,1.000,1.000,1.000


In [25]:
unlisted_X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15138 entries, 7716 to 12115
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   회사명                15138 non-null  object 
 1   거래소코드              15138 non-null  int64  
 2   회계년도               15138 non-null  int64  
 3   CASH FLOW 대 부채비율   15138 non-null  float64
 4   CASH FLOW 대 차입금비율  15138 non-null  float64
 5   CASH FLOW 대 총자본비율  15138 non-null  float64
 6   CASH FLOW 대 매출액비율  15138 non-null  float64
 7   유동비율               15138 non-null  float64
 8   당좌비율               15138 non-null  float64
 9   부채비율               15138 non-null  float64
 10  유동부채비율             15138 non-null  float64
 11  차입금의존도             15138 non-null  float64
 12  순운전자본비율            15138 non-null  float64
 13  현금비율               15138 non-null  float64
 14  유동자산구성비율           15138 non-null  float64
 15  자기자본구성비율           15138 non-null  float64
 16  자기자본순이익률           15138

In [26]:
unlisted_X_train.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [27]:
unlisted_X_train.isna().sum().sort_values(ascending=False)

회사명                  0
매출채권회전률              0
비유동자산증가율             0
매출액증가율               0
유형자산증가율              0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
당좌자산회전률              0
거래소코드                0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유동자산증가율              0
총자본증가율               0
총자산증가율               0
CROE                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율             0
자기자본구성비율             0
자기자본순이익률             0
경영자본순이익률   

In [28]:
unlisted_X_train.isin([0]).sum().sort_values(ascending=False)

성장기                  12246
도입기                  11837
쇠퇴기                  11765
부실판단                  9668
성숙기                   9566
CROE                  1860
자본분배율                 1598
이윤분배율                 1388
총자산회전율                1354
총자산증가율                1353
차입금의존도                1273
CASH FLOW 대 차입금비율     1268
매출채권회전률               1181
자기자본순이익률               755
자기자본구성비율               601
유동부채비율                 599
부채비율                   599
유형자산증가율                348
설비투자효율                 216
유형자산회전율                189
매출액증가율                 145
현금비율                    90
금융비용부담률                 83
총자본회전률                  54
비유동자산증가율                46
경영자본회전률                 37
비유동자산회전률                28
영업년수                    16
유동자산회전률                 14
총자본사업이익률                 9
총자본증가율                   8
CASH FLOW 대 총자본비율        8
CASH FLOW 대 매출액비율        8
매출액영업이익률                 8
총자본영업이익률                 8
경영자본순이익률                 7
CASH FLOW 대 부채비율         5
매

In [29]:
unlisted_X_train.isin([np.inf]).sum().sort_values(ascending=False)

총자산증가율               1
매출채권회전률              0
비유동자산증가율             0
매출액증가율               0
유형자산증가율              0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
회사명                  0
유동자산증가율              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
당좌자산회전률              0
총자본증가율               0
거래소코드                0
CROE                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율             0
자기자본구성비율             0
자기자본순이익률             0
경영자본순이익률   

In [30]:
inf_row = unlisted_X_train.loc[unlisted_X_train['총자산증가율'] == np.inf].index
unlisted_X_train.drop(inf_row, axis=0, inplace=True)

1현금흐름 4가지 cash

In [31]:
cash_ft1.describe()

,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000
mean,19.314,724.234,3.241,-22.741,0.361
std,1261.272,31970.320,37.792,539.239,0.480
min,-86467.650,-323703.660,-3388.620,-21110.050,0.000
25%,-4.480,-5.120,-2.880,-4.890,0.000
50%,2.920,1.510,1.860,2.360,0.000
75%,14.328,17.380,7.387,10.777,1.000
max,113999.840,3194164.340,1274.750,21579.540,1.000


2안정성 9가지 stability

In [32]:
stability_ft1.describe()

,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,431.245,358.475,1378.936,983.416,43.176,0.167,98.637,45.603,38.894,0.361
std,5347.663,5224.565,35986.322,30885.126,211.937,297.974,2442.606,28.781,26.118,0.480
min,0.070,0.070,0.000,0.000,0.000,-36261.200,-0.200,0.000,0.000,0.000
25%,56.050,31.620,60.845,36.860,17.360,-19.517,1.590,21.870,18.270,0.000
50%,108.715,72.700,163.855,102.565,42.040,2.925,8.605,43.180,35.145,0.000
75%,197.823,150.323,373.438,239.108,61.450,26.938,32.528,67.630,57.267,1.000
max,447760.110,447760.110,3385006.150,2905990.690,25856.360,99.690,275285.960,100.000,99.980,1.000


3수익성 8가지 profitability

In [33]:
productivity_ft1.describe()

,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,6674.974,30.528,538176.590,46.170,-95.676,8.411,0.361
std,352176.022,1669.910,34787344.705,747.415,2660.033,1074.584,0.480
min,-68167.780,-246.020,-563419210.000,-13763.270,-208392.960,-94627.640,0.000
25%,937.122,4.870,9.130,7.960,-0.310,22.102,0.000
50%,2281.575,11.060,28.820,16.220,11.390,55.870,0.000
75%,4883.170,19.218,97.688,33.188,35.943,74.770,1.000
max,43327598.520,205392.780,4053778655.000,70947.820,8226.360,126.190,1.000


4성장성 6개 growth

In [34]:
growth_ft1.describe()

,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,inf,6.597,216.233,302.142,228.063,192381.296,0.361
std,NaN,55.243,20598.824,11247.279,9908.536,21029765.042,0.480
min,-100.000,-99.890,-99.900,-100.000,-99.910,-100.000,0.000
25%,-4.447,-5.480,-14.060,-5.207,-12.510,-5.630,0.000
50%,0.523,1.820,3.990,-0.110,3.610,-1.040,0.000
75%,11.191,12.805,25.038,12.480,21.797,8.002,1.000
max,inf,5081.220,2531488.340,1154781.210,900275.000,2580453089.000,1.000


5생산성 6가지 productivity

In [35]:
productivity_ft1.describe()

,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,6674.974,30.528,538176.590,46.170,-95.676,8.411,0.361
std,352176.022,1669.910,34787344.705,747.415,2660.033,1074.584,0.480
min,-68167.780,-246.020,-563419210.000,-13763.270,-208392.960,-94627.640,0.000
25%,937.122,4.870,9.130,7.960,-0.310,22.102,0.000
50%,2281.575,11.060,28.820,16.220,11.390,55.870,0.000
75%,4883.170,19.218,97.688,33.188,35.943,74.770,1.000
max,43327598.520,205392.780,4053778655.000,70947.820,8226.360,126.190,1.000


6활동성 8가지 activity

In [36]:
activity_ft1.describe()

,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,0.938,700.636,4.246,7686.666,1.119,0.996,22.988,2.830,0.361
std,1.663,46090.572,7.518,263256.703,1.590,1.404,375.217,5.156,0.480
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.162,3.990,1.460,0.650,0.340,0.280,0.490,1.060,0.000
50%,0.652,7.150,2.880,2.060,0.820,0.730,1.460,1.990,0.000
75%,1.231,13.370,4.940,7.050,1.430,1.300,3.757,3.340,1.000
max,100.540,5386048.240,344.060,19823298.290,106.800,92.050,32610.280,344.060,1.000


7기타 etc

In [37]:
etc_ft1.describe()

,영업년수,도입기,성숙기,성장기,쇠퇴기,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,21.322,0.218,0.368,0.191,0.223,0.361
std,11.151,0.413,0.482,0.393,0.416,0.480
min,0.000,0.000,0.000,0.000,0.000,0.000
25%,13.000,0.000,0.000,0.000,0.000,0.000
50%,20.000,0.000,0.000,0.000,0.000,0.000
75%,27.000,0.000,1.000,0.000,0.000,1.000
max,110.000,1.000,1.000,1.000,1.000,1.000


# 상관분석과 유의성 검정

In [ ]:
# 상관 계수와 p-value 계산하는 함수
def correlation_and_pval(df):
    columns = df.columns
    corr_dict = {}
    pval_dict = {}
    
    # 모든 특성 쌍에 대한 상관 계수와 p-value 계산
    for col1, col2 in combinations(columns, 2):
        corr, pval = pearsonr(df[col1], df[col2])
        corr_dict[f"{col1}-{col2}"] = corr
        pval_dict[f"{col1}-{col2}"] = pval
    
    return corr_dict, pval_dict

1현금흐름

In [ ]:
plt.figure(figsize=(5,3))
mask = np.triu(np.ones_like(cash_ft1.corr(), dtype=bool))
sns.heatmap(cash_ft1.corr(),
            vmax=1,
            vmin=-1,
            # center=0,
            annot=True,
            annot_kws={'size' : 10}, # 숫자 출력 시 숫자 크기 조절
            fmt = '.3f', # 숫자의 출력 소수점자리 개수 조절
            square = True, # 차트를 정사각형으로 할 것인지
            cmap='bwr', 
            linewidths=.5, 
            mask = mask) # 좌하방만 표시

In [ ]:
# 안정성 피처 상관계수 및 p-value
# 상관 계수와 p-value 확인
correlations, p_values = correlation_and_pval(cash_ft1)

print("상관 계수======================")
# 결과 출력 (상관 계수)
for key, value in correlations.items():
    print(f"{key}: {value:.3f}")

print("p-value======================")
# 결과 출력 (p-value)
for key, value in p_values.items():
    print(f"{key}: {value:.3f}")

In [ ]:
np.abs(cash_ft1.corr()['부실판단']).sort_values(ascending=False)

2안정성

In [ ]:
plt.figure(figsize=(10,8))
mask = np.triu(np.ones_like(stability_ft1.corr(), dtype=bool))
sns.heatmap(stability_ft1.corr(), 
            vmax=1,
            vmin=-1,
            annot=True,
            annot_kws={'size' : 10}, # 숫자 출력 시 숫자 크기 조절
            fmt = '.3f', # 숫자의 출력 소수점자리 개수 조절
            square = True, # 차트를 정사각형으로 할 것인지
            cmap='bwr', 
            linewidths=.5, 
            mask = mask)

In [ ]:
# 안정성 피처 상관계수 및 p-value
# 상관 계수와 p-value 확인
correlations, p_values = correlation_and_pval(stability_ft1)

print("상관 계수======================")
# 결과 출력 (상관 계수)
for key, value in correlations.items():
    print(f"{key}: {value:.3f}")

print("p-value======================")
# 결과 출력 (p-value)
for key, value in p_values.items():
    print(f"{key}: {value:.3f}")

In [ ]:
np.abs(stability_ft1.corr()['부실판단']).sort_values(ascending=False)

3수익성

In [ ]:
plt.figure(figsize=(10,8))
mask = np.triu(np.ones_like(profitability_ft1.corr(), dtype=bool))
sns.heatmap(profitability_ft1.corr(),
            vmax=1,
            vmin=-1,
            annot=True,
            annot_kws={'size' : 10}, # 숫자 출력 시 숫자 크기 조절
            fmt = '.3f', # 숫자의 출력 소수점자리 개수 조절
            square = True, # 차트를 정사각형으로 할 것인지
            cmap='bwr', 
            linewidths=.5, 
            mask = mask)

In [ ]:
# 안정성 피처 상관계수 및 p-value
# 상관 계수와 p-value 확인
correlations, p_values = correlation_and_pval(profitability_ft1)

print("상관 계수======================")
# 결과 출력 (상관 계수)
for key, value in correlations.items():
    print(f"{key}: {value:.3f}")

print("p-value======================")
# 결과 출력 (p-value)
for key, value in p_values.items():
    print(f"{key}: {value:.3f}")

In [ ]:
np.abs(profitability_ft1.corr()['부실판단']).sort_values(ascending=False)

4성장성

In [ ]:
plt.figure(figsize=(10,8))
mask = np.triu(np.ones_like(growth_ft2.corr(), dtype=bool))
sns.heatmap(growth_ft2.corr(),
            vmax=1,
            vmin=-1,
            annot=True,
            annot_kws={'size' : 10}, # 숫자 출력 시 숫자 크기 조절
            fmt = '.3f', # 숫자의 출력 소수점자리 개수 조절
            square = True, # 차트를 정사각형으로 할 것인지
            cmap='bwr', 
            linewidths=.5, 
            mask = mask)

In [ ]:
# 안정성 피처 상관계수 및 p-value
# 상관 계수와 p-value 확인
correlations, p_values = correlation_and_pval(growth_ft2)

print("상관 계수======================")
# 결과 출력 (상관 계수)
for key, value in correlations.items():
    print(f"{key}: {value:.3f}")

print("p-value======================")
# 결과 출력 (p-value)
for key, value in p_values.items():
    print(f"{key}: {value:.3f}")

In [ ]:
np.abs(growth_ft2.corr()['부실판단']).sort_values(ascending=False)

5생산성

In [ ]:
plt.figure(figsize=(10,8))
mask = np.triu(np.ones_like(productivity_ft1.corr(), dtype=bool))
sns.heatmap(productivity_ft1.corr(), 
            vmax=1,
            vmin=-1,
            annot=True,
            annot_kws={'size' : 10}, # 숫자 출력 시 숫자 크기 조절
            fmt = '.3f', # 숫자의 출력 소수점자리 개수 조절
            square = True, # 차트를 정사각형으로 할 것인지
            cmap='bwr', 
            linewidths=.5, 
            mask = mask)

In [ ]:
# 안정성 피처 상관계수 및 p-value
# 상관 계수와 p-value 확인
correlations, p_values = correlation_and_pval(productivity_ft1)

print("상관 계수======================")
# 결과 출력 (상관 계수)
for key, value in correlations.items():
    print(f"{key}: {value:.3f}")

print("p-value======================")
# 결과 출력 (p-value)
for key, value in p_values.items():
    print(f"{key}: {value:.3f}")

In [ ]:
np.abs(productivity_ft1.corr()['부실판단']).sort_values(ascending=False)

6활동성

In [ ]:
plt.figure(figsize=(10,8))
mask = np.triu(np.ones_like(activity_ft1.corr(), dtype=bool))
sns.heatmap(activity_ft1.corr(), 
            vmax=1,
            vmin=-1,
            annot=True,
            annot_kws={'size' : 8}, # 숫자 출력 시 숫자 크기 조절
            fmt = '.3f', # 숫자의 출력 소수점자리 개수 조절
            square = True, # 차트를 정사각형으로 할 것인지
            cmap='bwr', 
            linewidths=.5, 
            mask = mask)

In [ ]:
# 안정성 피처 상관계수 및 p-value
# 상관 계수와 p-value 확인
correlations, p_values = correlation_and_pval(activity_ft1)

print("상관 계수======================")
# 결과 출력 (상관 계수)
for key, value in correlations.items():
    print(f"{key}: {value:.3f}")

print("p-value======================")
# 결과 출력 (p-value)
for key, value in p_values.items():
    print(f"{key}: {value:.3f}")

In [ ]:
np.abs(activity_ft1.corr()['부실판단']).sort_values(ascending=False)

7기타

In [ ]:
plt.figure(figsize=(10,8))
mask = np.triu(np.ones_like(etc_ft1.corr(), dtype=bool))
sns.heatmap(etc_ft1.corr(), 
            vmax=1,
            vmin=-1,
            annot=True,
            annot_kws={'size' : 8}, # 숫자 출력 시 숫자 크기 조절
            fmt = '.3f', # 숫자의 출력 소수점자리 개수 조절
            square = True, # 차트를 정사각형으로 할 것인지
            cmap='bwr', 
            linewidths=.5, 
            mask = mask
)

In [ ]:
# 안정성 피처 상관계수 및 p-value
# 상관 계수와 p-value 확인
correlations, p_values = correlation_and_pval(etc_ft1)

print("상관 계수======================")
# 결과 출력 (상관 계수)
for key, value in correlations.items():
    print(f"{key}: {value:.3f}")

print("p-value======================")
# 결과 출력 (p-value)
for key, value in p_values.items():
    print(f"{key}: {value:.3f}")

In [ ]:
np.abs(etc_ft1.corr()['부실판단']).sort_values(ascending=False)

전체

In [ ]:
plt.figure(figsize=(10,8))
mask = np.triu(np.ones_like(features2.corr(), dtype=bool))
sns.heatmap(features2.corr(), 
            vmax=1,
            vmin=-1,
            # annot=True,
            # annot_kws={'size' : 8}, # 숫자 출력 시 숫자 크기 조절
            # fmt = '.3f', # 숫자의 출력 소수점자리 개수 조절
            square = True, # 차트를 정사각형으로 할 것인지
            cmap='bwr', 
            linewidths=.1, 
            mask = mask
)

# QQplot

## before Q-Qplot

In [ ]:
def qq_plot(df):
    fig, axes = plt.subplots(8, 6, figsize=(20, 25))
    fig.subplots_adjust(wspace=0.5, hspace=0.5)  # 서브플롯 간 간격 조정
    
    for i in range(8):
        for j in range(6):
            col_index = i * 6 + j
            if col_index < len(df.columns):
                # Q-Q plot 그리기
                stats.probplot(df[df.columns[col_index]], dist="norm", plot=axes[i, j])
                axes[i, j].set_title(f"{df.columns[col_index]}")
            else:
                # 데이터프레임 칼럼 수보다 많은 서브플롯은 숨김
                axes[i, j].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
qq_plot(features)

## 윈저라이징

In [ ]:
# 한 컬럼마다 할 수 있는 윈저라이징
def winsorize(data, lower_percentile=5, upper_percentile=95):
    """
    이상치를 Winsorizing하여 데이터를 변형하는 함수
    
    Parameters:
    - data: 변형하려는 데이터 배열
    - lower_percentile: 이상치를 대체할 하한 백분위수
    - upper_percentile: 이상치를 대체할 상한 백분위수
    
    Returns:
    - winsorized_data: Winsorizing이 적용된 데이터 배열
    """
    lower_limit = np.percentile(data, lower_percentile)
    upper_limit = np.percentile(data, upper_percentile)
    
    daco = data.copy()
    # 하한 이상치 대체
    daco[daco < lower_limit] = lower_limit
    
    # 상한 이상치 대체
    daco[daco > upper_limit] = upper_limit
    
    return daco

In [ ]:
# 모든 컬럼을 일괄 윈저라이징
def adj_outlier(df, target=0.01, winsorizing=True):
    """
    데이터프레임에서 이상치를 조정합니다.

    매개변수:
    - dataframe: pandas DataFrame
    - 목표값: float, 이상치 식별을 위한 임계값
    - 윈저라이징: bool, 윈저라이징 기법을 적용할지 여부

    반환값:
    - 이상치가 조정된 pandas DataFrame
    """
    # 원본 DataFrame을 직접 수정하지 않기 위해 복사합니다.
    df_copy = df.copy()

    # 목표값에 기반하여 이상치를 식별합니다.    
    lower_bound = df_copy.quantile(target)
    upper_bound = df_copy.quantile(1 - target)

    # 윈저라이징이 지정된 경우 해당 열에 대해 이상치를 조정합니다.
    if winsorizing:
        df_copy = df_copy.apply(lambda x: np.clip(x, lower_bound[x.name], upper_bound[x.name]))

    return df_copy

# 예제 사용법:
# '당신의_데이터프레임'이 실제 데이터프레임인 경우
# 조정된_데이터프레임 = 이상치_조정(당신의_데이터프레임)

## 일괄 Q-Qplot

In [ ]:
adj_features = adj_outlier(features, target=0.01, winsorizing=True)
qq_plot(adj_features)

## 컬럼별 윈저라이징

In [ ]:
winsorized_features = features.copy()

In [ ]:
i = 0
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 1
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 2
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 3
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 4
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 5
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 6
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 7
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 8
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 9
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.99)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 10
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 11
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 12
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=100)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 13
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 14
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 15
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 16
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 17
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 18
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.99)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 19
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 20
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 21
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.01, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 22
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 23
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 24
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 25
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 26
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 27
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 28
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 29
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 30
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 31
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.99)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 32
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0.1, upper_percentile=99.99)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 33
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 34
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 35
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 36
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 37
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 38
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 39
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 40
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

In [ ]:
i = 41
col = features.columns[i]
raw_data = features[col]
winsorized_data = winsorize(raw_data, lower_percentile=0, upper_percentile=99.9)
winsorized_features[col] = winsorized_data
stats.probplot(winsorized_data, dist="norm", plot=plt)
plt.title(col)
plt.show()

## after QQplot

In [ ]:
qq_plot(winsorized_features)

# to_csv

train set

In [ ]:
unlisted_X_train[features.columns] = winsorized_features

In [ ]:
unlisted_X_train

In [ ]:
unlisted_X_train.to_csv('./unlisted_winsorized_train_data.csv', index=False)

test set

In [ ]:
unlisted_X_test['부실판단'] = unlisted_y_test
unlisted_X_test['부실판단']

In [ ]:
unlisted_X_test.to_csv('./unlisted_winsorized_test_data.csv', index=False)

In [ ]:
unlisted_X_test